In [14]:
# Import Modules
import math
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from warnings import catch_warnings
from warnings import simplefilter
import scipy.stats as stats
import itertools
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

In [15]:
# Data Treatment
data_path = "C:/Users/USER/Documents/workspace/BMED/DB_bench_BMED_for_LA.xlsx"
raw_Data= pd.read_excel(data_path,sheet_name="data_for_ML")

MVs = raw_Data[['T_operation','V_operation','E_operation','C_F_LA','C_A_LA']]
CVs = raw_Data[['CE','J_LA']]
X = MVs.values
Y = CVs.values

In [16]:
# evaluation function
def eval(y):
	return ((y[0] - 87.5)/87.5)**2 + ((y[1] - 76)/76)**2

In [17]:
# surrogate or approximation for the objective function
def surrogate(model, X):
    # catch any warning generated when making a prediction
    with catch_warnings():
        # ignore generated warnings
        simplefilter('ignore')
        return model.predict(X, return_std=True)

In [18]:
# Expected improvement acquisition function
def acquisition(X, Xsamples, model):
    # calculate the best surrgate score found so far
    yhat, _ = surrogate(model, X)
    score1 = np.zeros(len(yhat))
    for i in range(len(yhat)):
        score1[i] = eval(yhat[i])
    best = min(score1)

    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)
    score2 = np.zeros(len(mu))
    for i in range(len(mu)):
        score2[i] = eval(mu[i])
    # Calculate the expected improvement (EI)
    # Clip std to avoid division by zero
    std = np.clip(std, 1e-9, None)  # Replace None with a suitable upper bound if needed
    std2 = [i[0] for i in std]
    z = (score2 - best)**2 / std2
    ei = (score2 - best)**2 * stats.norm.cdf(z) + std2 * stats.norm.pdf(z)
    return ei

In [19]:
# optimize the acquisition function
def opt_acquisition(X, y, model):
    # random search, generate random samples
    Xs1 = 14 + 3*np.random.random(46)
    Xs2 = 1+0.2*np.random.random(46)
    Xs3 = -0.8 + 0.4*np.random.random(46)
    Xsamples = np.asarray(list(itertools.product(Xs1,Xs2,Xs3)))
    
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = np.argmin(scores)
    return Xsamples[ix]

In [20]:
# plot real observation vs surrogate function
def plot(X, y, model):
    # Split inputs
    Xe1 = [X[i][0] for i in range(len(X))]
    Xe2 = [X[i][1] for i in range(len(X))]
    Xe3 = [X[i][2] for i in range(len(X))]
    ye1 = [y[i][0] for i in range(len(y))]
    ye2 = [y[i][1] for i in range(len(y))]

    # scatter plot of imputs
    fig, axes = plt.subplots(2,3)
    axes[0,0].scatter(Xe1, ye1)
    axes[0,1].scatter(Xe2, ye1)
    axes[0,2].scatter(Xe3, ye1)
    axes[1,0].scatter(Xe1, ye2)
    axes[1,1].scatter(Xe2, ye2)
    axes[1,2].scatter(Xe3, ye2)

    # line plot of surragte function acorss domain
    Xp1 = np.append(Xe1,10 + 10*np.random.random(20))
    Xp2 = np.append(Xe2,2*np.random.random(20))
    Xp3 = np.append(Xe3,-10 + 20*np.random.random(20))
    Xpred = np.asarray(list(itertools.product(Xp1,Xp2,Xp3)))

    ypred, _ = surrogate(model, Xpred)
    yp1 = [i[0] for i in ypred]
    yp2 = [i[1] for i in ypred]

    Xpl1 = [i[0] for i in Xpred]
    Xpl2 = [i[1] for i in Xpred]
    Xpl3 = [i[2] for i in Xpred]

    axes[0,0].scatter(Xpl1, yp1, s=1)
    axes[0,1].scatter(Xpl2, yp1, s=1)
    axes[0,2].scatter(Xpl3, yp1, s=1)
    axes[1,0].scatter(Xpl1, yp2, s=1)
    axes[1,1].scatter(Xpl2, yp2, s=1)
    axes[1,2].scatter(Xpl3, yp2, s=1)
    # show the plot
    plt.show()

In [21]:
# split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

# 5-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)


for train_index, val_index in kfold.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[val_index]

    model = GaussianProcessRegressor()
    model.fit(X_train_fold,Y_train_fold)



# # define the model
# model = GaussianProcessRegressor()
# model.fit(X,y)

# x = opt_acquisition(X, y, model)
# actual = [67.06,49.637]
# est, _ = surrogate(model, [X[2]])
# print(f'>x = {x}, f()={est}, actual={actual}')
# print(np.exp(x[0]),np.exp(x[0])*5)


In [22]:
# best result
score3=np.zeros(len(y))
for i in range(len(y)):
        score3[i] = eval(y[i])
ix = np.argmin(score3)
print(X[ix], y[ix])

plt.scatter(score3)

,[16.0505649,1.19114408,-0.68813221]
,[67.19,49.637]

<>:10: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?
<>:11: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?
<>:10: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?
<>:11: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?


NameError: name 'y' is not defined